In [1]:
from gcn.inits import *
from gcn.utils import *
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# flags = tf.compat.v1.flags
# FLAGS = flags.FLAGS

2023-05-06 21:01:42.751941: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


## 1. Layers Class

### 1.1. Helper Functions

In [2]:
# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}


def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def sparse_dropout(x, keep_prob, noise_shape):
    """Dropout for sparse tensors."""
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1./keep_prob)


def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res

### 1.2. Layer Class

In [3]:
class Layer(object):
    """Base layer class. Defines basic API for all layer objects.
    Implementation inspired by keras (http://keras.io).

    # Properties
        name: String, defines the variable scope of the layer.
        logging: Boolean, switches Tensorflow histogram logging on/off

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
        _log_vars(): Log all variables
    """

    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.sparse_inputs = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            if self.logging and not self.sparse_inputs:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])

### 1.3. Dense Layer

In [4]:
class Dense(Layer):
    """Dense layer."""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0., sparse_inputs=False,
                 act=tf.nn.relu, bias=False, featureless=False, **kwargs):
        super(Dense, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless
        self.bias = bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = glorot([input_dim, output_dim],
                                          name='weights')
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # transform
        output = dot(x, self.vars['weights'], sparse=self.sparse_inputs)

        # bias
        if self.bias:
            output += self.vars['bias']

        return self.act(output)

### 1.4 Graph Convolutional Layer

In [5]:
class GraphConvolution(Layer):
    """Graph convolution layer."""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False,
                 featureless=False, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless
        self.bias = bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot([input_dim, output_dim],
                                                        name='weights_' + str(i))
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # convolve
        supports = list()
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x, self.vars['weights_' + str(i)],
                              sparse=self.sparse_inputs)
            else:
                pre_sup = self.vars['weights_' + str(i)]
            support = dot(self.support[i], pre_sup, sparse=True)
            supports.append(support)
        output = tf.add_n(supports)

        # bias
        if self.bias:
            output += self.vars['bias']

        return self.act(output)

## 2. Models

In [6]:
from gcn.metrics import *

# flags = tf.compat.v1.flags
# FLAGS = flags.FLAGS

In [7]:
class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None

        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)

In [8]:
class MLP(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(MLP, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=flags_learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += flags_weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):
        self.layers.append(Dense(input_dim=self.input_dim,
                                 output_dim=flags_hidden1,
                                 placeholders=self.placeholders,
                                 act=tf.nn.relu,
                                 dropout=True,
                                 sparse_inputs=True,
                                 logging=self.logging))

        self.layers.append(Dense(input_dim=flags_hidden1,
                                 output_dim=self.output_dim,
                                 placeholders=self.placeholders,
                                 act=lambda x: x,
                                 dropout=True,
                                 logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)

In [9]:
class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = flags_optimizer(learning_rate=flags_learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += flags_weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):

#         Paper layer configuration
        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=flags_hidden1,
                                            placeholders=self.placeholders,
                                            act=flags_act_func,
                                            dropout=True,
                                            sparse_inputs=True,
                                            logging=self.logging))

        self.layers.append(GraphConvolution(input_dim=flags_hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x,
                                            dropout=True,
                                            logging=self.logging))
#         Single layer configuration
#         self.layers.append(GraphConvolution(input_dim=self.input_dim,
#                                             output_dim=self.output_dim,
#                                             placeholders=self.placeholders,
#                                             act=flags_act_func,
#                                             dropout=True,
#                                             sparse_inputs=True,
#                                             logging=self.logging))
#         Triple layer configuration
#         self.layers.append(GraphConvolution(input_dim=self.input_dim,
#                                             output_dim=64,
#                                             placeholders=self.placeholders,
#                                             act=flags_act_func,
#                                             dropout=True,
#                                             sparse_inputs=True,
#                                             logging=self.logging))

#         self.layers.append(GraphConvolution(input_dim=64,
#                                             output_dim=flags_hidden1,
#                                             placeholders=self.placeholders,
#                                             act=flags_act_func,
#                                             dropout=True,
#                                             logging=self.logging))

#         self.layers.append(GraphConvolution(input_dim=flags_hidden1,
#                                             output_dim=self.output_dim,
#                                             placeholders=self.placeholders,
#                                             act=lambda x: x,
#                                             dropout=True,
#                                             logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)

## 3. Train

In [10]:
from __future__ import division
from __future__ import print_function

import time
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)


### 3.1 Training Parameters

In [11]:
# Settings
# flags = tf.compat.v1.flags
# FLAGS = flags.FLAGS
flags_dataset = 'cora'# , 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
flags_model = 'gcn' #, 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags_learning_rate = 0.01 #, 'Initial learning rate.')
flags_epochs = 200 #, 'Number of epochs to train.')
flags_hidden1 = 16 #, 'Number of units in hidden layer 1.')
flags_dropout = 0.5 #, 'Dropout rate (1 - keep probability).')
flags_weight_decay = 5e-4 #, 'Weight for L2 loss on embedding matrix.')
flags_early_stopping = 10 #, 'Tolerance for early stopping (# of epochs).')
flags_max_degree = 3 #, 'Maximum Chebyshev polynomial degree.')
flags_act_func = tf.nn.relu # Activation function: tf.nn.relu, tf.nn.leaky_relu, tf.nn.sigmoid, tf.nn.tanh, tf.nn.elu
flags_optimizer = tf.train.AdamOptimizer # Optimizer: tf.train.AdamOptimizer, tf.train.GradientDescentOptimizer, tf.train.AdadeltaOptimizer, tf.train.RMSPropOptimizer

### 3.2 Load Data and Train Model

In [12]:
# Load data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(flags_dataset)

# Some preprocessing
features = preprocess_features(features)
if flags_model == 'gcn':
    support = [preprocess_adj(adj)]
    num_supports = 1
    model_func = GCN
elif flags_model == 'gcn_cheby':
    support = chebyshev_polynomials(adj, flags_max_degree)
    num_supports = 1 + flags_max_degree
    model_func = GCN
elif flags_model == 'dense':
    support = [preprocess_adj(adj)]  # Not used
    num_supports = 1
    model_func = MLP
else:
    raise ValueError('Invalid argument for model: ' + str(flags_model))

# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
sess = tf.Session()


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

t_begin = time.time()
print("start training...")
# Train model
for epoch in range(flags_epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: flags_dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > flags_early_stopping and cost_val[-1] > np.mean(cost_val[-(flags_early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

print("total train time {:.5f}".format(time.time() - t_begin))

# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/Users/goutham/Downloads/cs598-dl-healthcare-proj-mainzhisheng/src/gcn/utils.py:70: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
2023-05-06 21:01:49.513476: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_7' with dtype int32
	 [[{{node Placeholder_7}}]]
2023-05-06 21:01:49.513594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_7' with dtype int32
	 [[{{node Placeholder_7}}]]
2023-05-06 21:01:49.522493: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) 

start training...
Epoch: 0001 train_loss= 1.95398 train_acc= 0.14286 val_loss= 1.94916 val_acc= 0.19200 time= 0.13065
Epoch: 0002 train_loss= 1.94717 train_acc= 0.24286 val_loss= 1.94463 val_acc= 0.25400 time= 0.01129
Epoch: 0003 train_loss= 1.93999 train_acc= 0.40000 val_loss= 1.93989 val_acc= 0.36000 time= 0.01132
Epoch: 0004 train_loss= 1.92993 train_acc= 0.60714 val_loss= 1.93522 val_acc= 0.38400 time= 0.01166
Epoch: 0005 train_loss= 1.92239 train_acc= 0.62143 val_loss= 1.93089 val_acc= 0.38400 time= 0.01158
Epoch: 0006 train_loss= 1.91587 train_acc= 0.62143 val_loss= 1.92688 val_acc= 0.38800 time= 0.01148
Epoch: 0007 train_loss= 1.90422 train_acc= 0.65714 val_loss= 1.92295 val_acc= 0.38800 time= 0.01170
Epoch: 0008 train_loss= 1.90035 train_acc= 0.60714 val_loss= 1.91907 val_acc= 0.37200 time= 0.01237
Epoch: 0009 train_loss= 1.88918 train_acc= 0.61429 val_loss= 1.91513 val_acc= 0.37200 time= 0.01292
Epoch: 0010 train_loss= 1.88187 train_acc= 0.62143 val_loss= 1.91109 val_acc= 0.36

Epoch: 0099 train_loss= 0.89164 train_acc= 0.95000 val_loss= 1.32282 val_acc= 0.78600 time= 0.01169
Epoch: 0100 train_loss= 0.83943 train_acc= 0.97143 val_loss= 1.31836 val_acc= 0.78400 time= 0.01159
Epoch: 0101 train_loss= 0.89123 train_acc= 0.92857 val_loss= 1.31348 val_acc= 0.78000 time= 0.01207
Epoch: 0102 train_loss= 0.84613 train_acc= 0.92857 val_loss= 1.30786 val_acc= 0.78200 time= 0.01316
Epoch: 0103 train_loss= 0.85106 train_acc= 0.94286 val_loss= 1.30189 val_acc= 0.78200 time= 0.01274
Epoch: 0104 train_loss= 0.83774 train_acc= 0.92857 val_loss= 1.29585 val_acc= 0.78600 time= 0.01252
Epoch: 0105 train_loss= 0.82232 train_acc= 0.96429 val_loss= 1.28971 val_acc= 0.78600 time= 0.01130
Epoch: 0106 train_loss= 0.84253 train_acc= 0.95714 val_loss= 1.28336 val_acc= 0.78600 time= 0.01124
Epoch: 0107 train_loss= 0.83385 train_acc= 0.95000 val_loss= 1.27727 val_acc= 0.78600 time= 0.01170
Epoch: 0108 train_loss= 0.83776 train_acc= 0.96429 val_loss= 1.27111 val_acc= 0.78600 time= 0.01130


Epoch: 0181 train_loss= 0.62059 train_acc= 0.95000 val_loss= 1.09067 val_acc= 0.79200 time= 0.03352
Epoch: 0182 train_loss= 0.62461 train_acc= 0.96429 val_loss= 1.08900 val_acc= 0.79000 time= 0.06612
Epoch: 0183 train_loss= 0.62334 train_acc= 0.96429 val_loss= 1.08723 val_acc= 0.79000 time= 0.04023
Epoch: 0184 train_loss= 0.65683 train_acc= 0.95000 val_loss= 1.08566 val_acc= 0.79000 time= 0.01302
Epoch: 0185 train_loss= 0.66517 train_acc= 0.96429 val_loss= 1.08404 val_acc= 0.79000 time= 0.01589
Epoch: 0186 train_loss= 0.68030 train_acc= 0.94286 val_loss= 1.08288 val_acc= 0.79000 time= 0.02886
Epoch: 0187 train_loss= 0.60279 train_acc= 0.95000 val_loss= 1.08196 val_acc= 0.79000 time= 0.01805
Epoch: 0188 train_loss= 0.60231 train_acc= 0.97857 val_loss= 1.08095 val_acc= 0.79000 time= 0.01425
Epoch: 0189 train_loss= 0.61390 train_acc= 0.96429 val_loss= 1.07952 val_acc= 0.79000 time= 0.01721
Epoch: 0190 train_loss= 0.59581 train_acc= 0.96429 val_loss= 1.07784 val_acc= 0.79000 time= 0.02521
